In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd


import pandas as pd
import os
# cell border experimental code: 

import pandas as pd
import numpy as np
from scipy.spatial import ConvexHull


In [ ]:
df1 = pd.read_csv()

In [3]:
# # test case position data for cd3 

# cd3_position = pd.read_csv('edge_test/tumor edge test CD3 position_Selection.csv' , skiprows=3)
# cd3_position.drop(columns=['Unit', 'Category',
#        'Collection', 'Time', 'CC3 catchall', 'CC3 real', 'CD103',
#        'CD103+ cDC1', 'CD3','CD177','CD3-CD68', 'CD3-CD68+CD206', 'CD4',
#        'CD45',  'Unnamed: 18'], inplace=True)

# cd3_distance = pd.read_csv('edge_test/tumor edge test_CD3 distance to tumor edge_Selection.csv' , skiprows=3)
# cd3_distance.drop(columns=['Category', 'Surfaces', 'Time',
#        'CC3 catchall', 'CC3 real', 'CD103', 'CD103+ cDC1', 'CD177', 'CD3',
#        'CD3-CD68', 'CD3-CD68+CD206', 'Unit','CD4', 'CD45', 'Unnamed: 16'], inplace=True)

# # cd3_position.head()
# cd3_final = cd3_position.merge(cd3_distance, on=["ID"], how='left').sort_values(by=['ID'], ascending=True)
# cd3_final =  cd3_final[['ID','Position X', 'Position Y', 'Position Z',
#        'Shortest Distance to Surfaces']].set_index('ID')
# cd3_final.head()

In [6]:
# Test case position data for dapi 

dapi2_position = pd.read_csv('edge_test/tumor edge test DAPI position_Selection.csv' , skiprows=3)
dapi2_position.drop(columns=['Unit', 'Category',
       'Collection', 'Time', 'CC3 catchall', 'CC3 real', 'CD103',
       'CD103+ cDC1', 'CD3','CD177','CD3-CD68', 'CD3-CD68+CD206', 'CD4',
       'CD45',  'Unnamed: 18'], inplace=True)

dapi2_distance = pd.read_csv('edge_test/tumor edge test_DAPI distance to tumor edge_Selection.csv' , skiprows=3)
dapi2_distance.drop(columns=['Category', 'Surfaces', 'Time',
       'CC3 catchall', 'CC3 real', 'CD103', 'CD103+ cDC1', 'CD177', 'CD3',
       'CD3-CD68', 'CD3-CD68+CD206', 'Unit','CD4', 'CD45', 'Unnamed: 16'], inplace=True)

# cd3_position.head()
dapi2_final = dapi2_position.merge(dapi2_distance, on=["ID"], how='left').sort_values(by=['ID'], ascending=True)
dapi2_final =  dapi2_final[['ID','Position X', 'Position Y', 'Position Z',
       'Shortest Distance to Surfaces']].set_index('ID')
dapi2_final.head()

/var/folders/bv/7wzy33ls6ln4ydwr6s8vwq8h0000gn/T/ipykernel_66185/4266110926.py:3: DtypeWarning: Columns (8,9,10,11,12,13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  dapi2_position = pd.read_csv('edge_test/tumor edge test DAPI position_Selection.csv' , skiprows=3)


,Position X,Position Y,Position Z,Shortest Distance to Surfaces
ID,,,,
0,87286.352,39678.461,2.373,256.589
1,87253.484,39703.367,6.589,279.778
2,87024.227,39933.770,6.702,174.885
3,86896.992,40064.652,5.901,55.385
4,87015.500,39945.977,5.861,164.362


In [10]:

# 1) Load your data
df =  dapi2_final # columns “x” and “y”
pts = df[['Position X', 'Position Y']].values             # shape (N,2) array

# 2) Compute the convex hull
hull = ConvexHull(pts)

# hull.simplices is an (E,2) array of index pairs (i, j)
# each representing an edge between pts[i] and pts[j].
edges = hull.simplices

# 3) Define a vectorized distance‐to‐segment function
def point_to_segment_distances(points, seg_start, seg_end):
    """
    points: (N,2) array
    seg_start, seg_end: each (2,) array
    returns: (N,) array of distances from each point to the segment
    """
    v = seg_end - seg_start                      # (2,)
    w = points - seg_start                       # (N,2)
    # projection factor of each point onto the infinite line
    t = np.einsum('ij,j->i', w, v) / np.dot(v, v)
    # clamp to [0,1] so we “snap” to the segment
    t_clamped = np.clip(t, 0.0, 1.0)             # (N,)
    proj = seg_start + t_clamped[:,None] * v     # (N,2)
    # euclidean distance from each point to its projection
    return np.linalg.norm(points - proj, axis=1)

# 4) Loop over edges, keep the minimum distance per point
all_dists = np.full(len(pts), np.inf)
for i, j in edges:
    start, end = pts[i], pts[j]
    d = point_to_segment_distances(pts, start, end)
    all_dists = np.minimum(all_dists, d)

# 5) Attach back to your DataFrame
df['dist_to_hull'] = all_dists

# 6) (Optional) Inspect
df

,Position X,Position Y,Position Z,Shortest Distance to Surfaces,dist_to_hull
ID,,,,,
0,87286.352,39678.461,2.373,256.589,344.303829
1,87253.484,39703.367,6.589,279.778,344.728350
2,87024.227,39933.770,6.702,174.885,227.469084
3,86896.992,40064.652,5.901,55.385,106.869526
4,87015.500,39945.977,5.861,164.362,216.583741
...,...,...,...,...,...
62183,87404.539,36997.043,2.713,40.845,83.511607
62184,87410.164,37034.875,0.963,78.880,121.603161
62185,87404.609,37051.000,5.010,93.720,137.361276
